In [1]:
%pip install transformers datasets
%pip install accelerate==0.27.2
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Importing libraries
from huggingface_hub import login, HfApi
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer, AutoProcessor, AutoFeatureExtractor
from datasets import load_dataset
from torchvision import transforms
import torch
import numpy as np
import gradio as gr
from PIL import Image
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# Loading the dataset
dataset = load_dataset("cifar10")

In [5]:
# Lets preprocess the dataset

def preprocess_function(examples):
    transform = transforms.Compose([
        transforms.ToTensor(),  # Convert images to PyTorch tensors
    ])

    examples['pixel_values'] = [transform(img.convert("RGB")) for img in examples['img']]
    return examples

In [6]:
# Lets apply the preprocessing to the dataset

dataset = dataset.map(preprocess_function, batched=True)

dataset.set_format(type='torch', columns=['pixel_values', 'label'])

In [7]:
# Lets split the dataset
train_dataset = dataset['train']
test_dataset = dataset['test']

In [8]:
# Loading the pre-trained ResNet-50 model along with the feature extractor
model = AutoModelForImageClassification.from_pretrained('microsoft/resnet-50', num_labels=10, ignore_mismatched_sizes=True)
model.to(device)

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([10, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

In [9]:
# Lets define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)

In [10]:
# Lets define the compute_metrics function
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = (pred == labels).mean()
    return {"accuracy": accuracy}

In [11]:
# Perform training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [12]:
# Train the model
trainer.train()

  0%|          | 0/7815 [00:00<?, ?it/s]

{'loss': 2.3018, 'grad_norm': 1.788678526878357, 'learning_rate': 4.9936020473448496e-05, 'epoch': 0.01}
{'loss': 2.3003, 'grad_norm': 2.0236809253692627, 'learning_rate': 4.9872040946896996e-05, 'epoch': 0.01}
{'loss': 2.2999, 'grad_norm': 3.057823419570923, 'learning_rate': 4.980806142034549e-05, 'epoch': 0.02}
{'loss': 2.2965, 'grad_norm': 2.470679998397827, 'learning_rate': 4.974408189379399e-05, 'epoch': 0.03}
{'loss': 2.2935, 'grad_norm': 3.7237322330474854, 'learning_rate': 4.968010236724249e-05, 'epoch': 0.03}
{'loss': 2.2943, 'grad_norm': 1.962981104850769, 'learning_rate': 4.961612284069098e-05, 'epoch': 0.04}
{'loss': 2.2901, 'grad_norm': 4.212790489196777, 'learning_rate': 4.9552143314139475e-05, 'epoch': 0.04}
{'loss': 2.2909, 'grad_norm': 4.214333534240723, 'learning_rate': 4.9488163787587976e-05, 'epoch': 0.05}
{'loss': 2.2875, 'grad_norm': 2.1708288192749023, 'learning_rate': 4.942418426103647e-05, 'epoch': 0.06}
{'loss': 2.2869, 'grad_norm': 3.139894962310791, 'learnin

  0%|          | 0/313 [00:00<?, ?it/s]

{'eval_loss': 1.9325177669525146, 'eval_accuracy': 0.4572, 'eval_runtime': 19.7574, 'eval_samples_per_second': 506.139, 'eval_steps_per_second': 15.842, 'epoch': 1.0}
{'loss': 1.8236, 'grad_norm': 9.707344055175781, 'learning_rate': 3.995521433141395e-05, 'epoch': 1.0}
{'loss': 1.7828, 'grad_norm': 18.951034545898438, 'learning_rate': 3.989123480486245e-05, 'epoch': 1.01}
{'loss': 1.7963, 'grad_norm': 18.667421340942383, 'learning_rate': 3.9827255278310945e-05, 'epoch': 1.02}
{'loss': 1.7683, 'grad_norm': 12.65884780883789, 'learning_rate': 3.976327575175944e-05, 'epoch': 1.02}
{'loss': 1.8086, 'grad_norm': 15.62231159210205, 'learning_rate': 3.969929622520794e-05, 'epoch': 1.03}
{'loss': 1.7642, 'grad_norm': 17.41957664489746, 'learning_rate': 3.963531669865643e-05, 'epoch': 1.04}
{'loss': 1.7534, 'grad_norm': 7.8288116455078125, 'learning_rate': 3.9571337172104924e-05, 'epoch': 1.04}
{'loss': 1.7924, 'grad_norm': 13.024662971496582, 'learning_rate': 3.9507357645553424e-05, 'epoch': 1

  0%|          | 0/313 [00:00<?, ?it/s]

...

  0%|          | 0/313 [00:00<?, ?it/s]

{'eval_loss': 1.0994808673858643, 'eval_accuracy': 0.6262, 'eval_runtime': 23.4254, 'eval_samples_per_second': 426.887, 'eval_steps_per_second': 13.362, 'epoch': 4.0}
{'loss': 1.224, 'grad_norm': 11.376456260681152, 'learning_rate': 9.948816378758798e-06, 'epoch': 4.01}
{'loss': 1.1037, 'grad_norm': 10.054109573364258, 'learning_rate': 9.884836852207294e-06, 'epoch': 4.01}
{'loss': 1.0879, 'grad_norm': 12.844393730163574, 'learning_rate': 9.820857325655791e-06, 'epoch': 4.02}
{'loss': 1.1737, 'grad_norm': 8.698099136352539, 'learning_rate': 9.756877799104288e-06, 'epoch': 4.02}
{'loss': 1.1187, 'grad_norm': 7.172879695892334, 'learning_rate': 9.692898272552783e-06, 'epoch': 4.03}
{'loss': 1.1641, 'grad_norm': 7.6466755867004395, 'learning_rate': 9.62891874600128e-06, 'epoch': 4.04}
{'loss': 1.2817, 'grad_norm': 7.0514349937438965, 'learning_rate': 9.564939219449778e-06, 'epoch': 4.04}
{'loss': 1.1687, 'grad_norm': 7.876298904418945, 'learning_rate': 9.500959692898273e-06, 'epoch': 4.05

  0%|          | 0/313 [00:00<?, ?it/s]

{'eval_loss': 1.1055949926376343, 'eval_accuracy': 0.6374, 'eval_runtime': 23.5101, 'eval_samples_per_second': 425.35, 'eval_steps_per_second': 13.313, 'epoch': 5.0}
{'train_runtime': 2917.5858, 'train_samples_per_second': 85.687, 'train_steps_per_second': 2.679, 'train_loss': 1.4445068436864852, 'epoch': 5.0}


TrainOutput(global_step=7815, training_loss=1.4445068436864852, metrics={'train_runtime': 2917.5858, 'train_samples_per_second': 85.687, 'train_steps_per_second': 2.679, 'total_flos': 1.08419429376e+17, 'train_loss': 1.4445068436864852, 'epoch': 5.0})

In [13]:
# Evaluate the model
trainer.evaluate()

  0%|          | 0/313 [00:00<?, ?it/s]

{'eval_loss': 1.1055949926376343,
 'eval_accuracy': 0.6374,
 'eval_runtime': 23.7607,
 'eval_samples_per_second': 420.863,
 'eval_steps_per_second': 13.173,
 'epoch': 5.0}

In [14]:
# Lets login to the Hugging Face account
login()

In [ ]:
# Lets save the model to the Hugging Face Hub
model_path = "./results/checkpoint-7500"
model = AutoModelForImageClassification.from_pretrained(model_path)

In [ ]:
# Load the pre-trained model from Hugging Face
model_name = "youzaina001/cifar10-resnet50"
model = AutoModelForImageClassification.from_pretrained(model_name).to(device)
  # Replace with the appropriate processor if available

# Manually define preprocessing for Gradio
def manual_preprocess(image):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
    ])
    return transform(image).unsqueeze(0).to(device)

In [ ]:
# Lets build a Gradio interface for the model
def classify_image(image):
    
    # Preprocess the image
    image = image.convert("RGB")
    inputs = manual_preprocess(image)
    inputs = {"pixel_values": inputs}

    # Predict the class of the image
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get the predicted class
    predicted_class_idx = logits.argmax(-1).item()
    predicted_class = model.config.id2label[predicted_class_idx]
    return predicted_class

In [ ]:
# Lets create a Gradio interface

gface = gr.Interface(
    fn=classify_image,
    inputs=gr.components.Image(type="pil", label="Input Image"),
    outputs=gr.components.Label(num_top_classes=3),
    title="ResNet-50-CIFAR-10 Image Classifier",
    description="A pre-trained ResNet-50 model fine-tuned on the CIFAR-10 dataset.",
)

In [ ]:
gface.launch()

In [ ]:
# Creating a shareable link to the Gradio interface
gr.Interface(classify_image, "image", "label").launch(share=True)

In [ ]:
# Lets upload the model to the Hugging Face Hub
api = HfApi()
api.create_repo(repo_id="youzaina001/cifar10-resnet50")
model.push_to_hub("youzaina001/cifar10-resnet50")